# Пример генерации текста рекурентной сетью

In [2]:
import sys
import random
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

## Загружаем исходные данные

In [3]:
with open('./data/anna_karenina.txt', 'rb') as src:
    binary = src.read()
    text = binary.decode('utf8').lower()
print('corpus length:', len(text))

corpus length: 1710408


## Векторизуем текстовый корпус

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), 
             dtype=np.bool)

y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 93
nb sequences: 570123
Vectorization...


## Строим модель

In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

Build model...
Instructions for updating:
Colocations handled automatically by placer.


## Возвращает индекс выбранного символа для вектора вероятностей

In [6]:
def sample(preds, temperature = 1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-32) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Запускаем обучение

In [ ]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Instructions for updating:
Use tf.cast instead.
570123/570123 [==============================] - 1948s 3ms/sample - loss: 1.9444

----- diversity: 0.2
----- Generating with seed: "шний доктор давал ей рыбий жир, потом же"
шний доктор давал ей рыбий жир, потом женался с меня и понять и не было подумала она с ворони постоянный только в комната и последнее возможно подомолла постоянный с то, что он не постором постоянно подолжал с вернулся и постоянный совершенный не поделать и на нестанить и не понять и подевал к ней и постояния и последнее когда она не было положения и понять и положения и не постоянный и не понял в постоянный посторил с передейно и подел

----- diversity: 0.5
----- Generating with seed: "шний доктор давал ей рыбий жир, потом же"
шний доктор давал ей рыбий жир, потом желания возможны коротом сказать понять и обратся только уже то, что дом облонского продолжал и не поведен и очень поняла его разговор, и вся 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ние с вави и все забыл и был обоих свет от всегда в как ты покачивая мих сказал сергей иванович все мало убести в чем не приливитела ничего было проставлял с нее привезу перебрать свет на своего столы проступел протовала с дро все последней на все он могу б

----- diversity: 1.0
----- Generating with seed: "ума свияжского.

кроме того, этот вопрос"
ума свияжского.

кроме того, этот вопросстка, никакием и испитатно, даме еюся опять своим мужи мне, видла, он их тем веселму и то все токой губить за в том зада, и перекреснилась за женить покадь михай доловила должно, здесь елевым и в замолку за условой. еду н ьбя были тровах. лучше деяли вмеща мига; она знают болымя им знал своим женщину, и их, не мою долда мужа. я делаю, когда степанса. его покажи, состой. ты пушись, столы такие свид

----- diversity: 1.2
----- Generating with seed: "ума свияжского.

кроме того, этот вопрос"
ума свияжского.

кроме того, этот вопрос почти оботв, обходиличию жеищее, как он раз ездъюми слышави. – старшайтара

## Предсказания на предобученной модели

In [7]:
model.load_weights('./text-lstm.hd5')

In [8]:
start_index = random.randint(0, len(text) - maxlen - 1)

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: " есть прекращения отношений с любовником"
 есть прекращения отношений с любовником стояли в петербурге простить свою ответил его в последно в соберно выразить его стал бы все подал его при нее стал стояли в том, что он все приглазы были не смотрел на него в том, что он понимают в себя было труда, он вспоминая на нее призн

KeyboardInterrupt: 